In [7]:
import serial
import time
import numpy as np
import glob
import threading

def find_usb_modem():
    # For macOS
    devices = glob.glob('/dev/tty.usbmodem*')
    # For Linux
    devices += glob.glob('/dev/ttyACM*')
    return devices[0] if devices else None

# Constants
SERIAL_PORT = find_usb_modem()
if not SERIAL_PORT:
    raise Exception("USB modem device not found!")
BAUD_RATE = 115200  # Change this to your servo drive's baud rate
TIMEOUT = 1  # Serial timeout in seconds
motor_data = []  # This list will act as our data frame

# Establish serial connection
ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=TIMEOUT)

In [8]:
def send_command(command):
    ser.write((command + '\n').encode())  # Append \n to the command

def read_from_serial():
    while True:
        line = ser.readline().decode().strip()
        if line:  # Check if line is not empty
            motor_data.append(line)

# Start the thread to read from serial
read_thread = threading.Thread(target=read_from_serial)
read_thread.start()

def parse_recorded_data(data):
    parsed_data = []
    current_target = None

    for line in data:
        if "unknown cmd err" in line:
            # Handle or log the error
            continue
        elif "Target:" in line:
            current_target = float(line.split(":")[1].strip())
        else:
            values = [float(val) for val in line.split('\t')]
            parsed_data.append({
                "_MON_TARGET": values[0],
                "_MON_VOLT_Q": values[1],
                "_MON_VEL": values[2],
                "_MON_ANGLE": values[3]
            })

    return parsed_data

In [10]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "browser"

# send_command("")

motor_data = []

send_command("Motion:angle")
send_command("M0")
time.sleep(1)
send_command("M1")
time.sleep(1)
send_command("M0")
time.sleep(1)

parsed_data = parse_recorded_data(motor_data)

# Extracting _MON_TARGET and _MON_ANGLE values
mon_target_values = [entry["_MON_TARGET"] for entry in parsed_data]
mon_angle_values = [entry["_MON_ANGLE"] for entry in parsed_data]

# Creating the interactive plot
fig = go.Figure()

# Adding _MON_TARGET trace
fig.add_trace(go.Scatter(y=mon_target_values, mode='lines', name='_MON_TARGET'))

# Adding _MON_ANGLE trace
fig.add_trace(go.Scatter(y=mon_angle_values, mode='lines', name='_MON_ANGLE', line=dict(dash='dash')))

# Setting plot title and labels
fig.update_layout(title='_MON_TARGET vs _MON_ANGLE', xaxis_title='Time Number', yaxis_title='Value')

# Displaying the plot
fig.show()



Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/narayanpowderly/.platformio/python3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/narayanpowderly/.platformio/python3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/xl/r_y6qlm504l70lmz99q_ndfw0000gn/T/ipykernel_96767/1778965355.py", line 6, in read_from_serial
  File "/Users/narayanpowderly/.platformio/penv/lib/python3.9/site-packages/serial/serialposix.py", line 595, in read
    raise SerialException(
serial.serialutil.SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)
